In [1]:
#this defaults plots to matplotlib 1.0 formatting
import matplotlib.style
import matplotlib as mpl
#mpl.style.use('classic')
%matplotlib inline
mpl.rcParams['figure.dpi']= 150

# importing packages needed to run this script 
from subprocess import call
import pickle as pickle
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.cluster.hierarchy as sch

import os
import time
import matplotlib.pyplot as plt
import numpy as np
import copy
import colorsys
import matplotlib.image as mpimg
from IPython.display import display
from IPython.display import Image
import scipy.cluster.hierarchy as sch
from scipy.stats import scoreatpercentile 
from pysca import scaTools as sca
# import mpld3
import pickle as pickle
from optparse import OptionParser
import sys
sys.path.append('../utilities/')

from graph_helpers import (
   gen_amino_acid_position_in_ref_sequence_graph,
   gen_clustered_similarity_matrix, 
   gen_unclustered_similarity_matrix,
   gen_pairwise_sequence_identities
)
from helpers import (
    show_processmsa_results,
    seq_count_check
)

%matplotlib inline

if not os.path.exists('../output/'):
    os.makedirs('../output/')  

In [2]:
input_pfam = '/home/bjohnson/rdb/data/trc8pfamalign.txt'
output_pfam = '/home/bjohnson/rdb/data/trc8pfamalign-annotated.txt'
pfam_seq = '/home/bjohnson/rdb/data/pfamseq.txt'
#construct a dictionary of phylogenetic groups
sca.AnnotPfam(input_pfam, output_pfam, pfam_seq=pfam_seq)
print('Executed')

Beginning annotation
Current step 0, key G3P4T2_GASAC
Current step 1, key A0A2P8XWF4_BLAGE
Current step 2, key A0A2K5DM88_AOTNA
Current step 3, key A0A668VLA0_OREAU
Current step 4, key A0A3Q2UYC4_HAPBU
Current step 5, key A0A091EBY4_CORBR
Current step 6, key A0A671QQB1_9TELE
Current step 7, key A0A5E4BGN9_MARMO
Current step 8, key A0A267GZH2_9PLAT
Current step 9, key G3IP89_CRIGR
Current step 10, key A0A093NJ05_PYGAD
Current step 11, key A0A402F1Y3_9SAUR
Current step 12, key A0A0N8JW12_SCLFO
Current step 13, key A0A3P8Z3K4_ESOLU
Current step 14, key A0A4W6DL80_LATCA
Current step 15, key A0A3B0KGS1_DROGU
Current step 16, key A0A4Z2HQE9_9TELE
Current step 17, key A0A3B1JMM1_ASTMX
Current step 18, key A0A267G4J2_9PLAT
Current step 19, key A0A2K5TYS4_MACFA
Current step 20, key L5LQT0_MYODS
Current step 21, key A0A667YMZ7_9TELE
Current step 22, key A0A2I2YII1_GORGO
Current step 23, key A0A2G9UKQ9_TELCI
Current step 24, key A0A673Y8J8_SALTR
Current step 25, key A0A2R8ZAF8_PANPA
Current step 

In [31]:
path2aln_processed = '/home/bjohnson/rdb/data/trc8pfamalign-annotated.txt'
path2out = '../output/'
command =  str('scaProcessMSA -a %s -d %s -i 0' % (path2aln_processed, path2out))
print(command)
call(command,shell = 'True')
print("Executed")

scaProcessMSA -a /home/bjohnson/rdb/data/trc8pfamalign-annotated.txt -d ../output/ -i 0
Loaded alignment of 1333 sequences, 1525 positions.
Checking alignment for non-standard amino acids
Aligment size after removing sequences with non-standard amino acids: 1307
Trimming alignment for highly gapped positions (80% or more).
1307
Alignment size post-trimming: 520 positions
Looking for PDBs in home/bjohnson/pySCA-master/data/
using provided reference index 0
G3P4T2_GASAC|Ring finger protein 139 |Gasterosteus aculeatus (Three-spined stickleback)|Eukaryota,Metazoa,Chordata,Craniata,Vertebrata,Euteleostomi,Actinopterygii,Neopterygii,Teleostei,Neoteleostei,Acanthomorphata,Eupercaria,Perciformes,Cottioidei,Gasterosteales,Gasterosteidae,Gasterosteus.
here!
len refseq 496, len refpos 520, len algseq 520, len pairalg 520, len gloalg 520
Conducting sequence and position filtering: alignment size is 1307 seqs, 520 pos
ATS should also have 520 positions - ATS: 520
Keeping 983 sequences of 1307 seque

In [32]:
dbname = "trc8pfamalign-annotated.db"
pickle_path = '/home/bjohnson/rdb/output/trc8pfamalign-annotated.db'
db = pickle.load(open(pickle_path,'rb'))
Dseq = db['sequence']

In [33]:
show_processmsa_results(Dseq)

*****Final processed alignment parameters****
Number of sequences: M = 939
Number of effective sequences: M' = 90
Number of alignment positions: L = 474
Number of positions in the ats: 474
Number of structure positions mapped: 474
Missing key distmat
****************************************


In [34]:
# check for sufficient sequences in alignment
seq_count_check(Dseq)

The number of effective sequences in this MSA is: 90
This alignment does not have enough effective sequences to confidently analyze with SCA.
Npos (L) = 474


In [35]:
# Apply scaCore to our processed alignment 
command = 'scaCore -i %s' % (pickle_path)
print (command)
call(command, shell = 'True')
db = pickle.load(open(pickle_path,'rb'))
Dsca = db['sca']
print("Executed")

scaCore -i /home/bjohnson/rdb/output/trc8pfamalign-annotated.db
Computing the sequence projections.
basicICA failed to converge: 33.681321661230896
Computing sequence similarity matrix.
Computing the SCA conservation and correlation values.
Computing matrix randomizations...
Randomizations complete, 10 trials, time: 2.0 minutes
Calculations complete, writing to database file /home/bjohnson/rdb/output/trc8pfamalign-annotated
Executed


In [36]:
# run scaSectorID 
command = 'scaSectorID -i %s' % (pickle_path)
print (command)
call(command, shell = 'True')

# Load and store results of scaSectorID
db = pickle.load(open(pickle_path,'rb'))
Dsect = db['sector'] 
print("Executed")

scaSectorID -i /home/bjohnson/rdb/output/trc8pfamalign-annotated.db
Selected kpos=10 significant eigenmodes.
Calculations complete, writing to database file /home/bjohnson/rdb/output/trc8pfamalign-annotated
Executed


In [37]:
annot = dict()
for i, h in enumerate(Dseq['hd']):
    hs = h.split('|')
    annot[hs[0]] = sca.Annot(hs[1], hs[2], hs[3].replace('.',''))

# Most frequent taxonomic groups:
atleast = 1
for level in range(4):
    descr_list = [a.taxo.split(',')[level] for a in annot.values() \
                if len(a.taxo.split(',')) > level]
    descr_dict = {k:descr_list.count(k) for k in descr_list \
                if descr_list.count(k)>=atleast}
    print('\n Level %i:' % level)
    print(descr_dict)


 Level 0:
{'Eukaryota': 939}

 Level 1:
{'Metazoa': 939}

 Level 2:
{'Chordata': 786, 'Ecdysozoa': 130, 'Cnidaria': 4, 'Spiralia': 16, 'Echinodermata': 3}

 Level 3:
{'Craniata': 783, 'Arthropoda': 130, 'Anthozoa': 4, 'Lophotrochozoa': 16, 'Cephalochordata': 3, 'Eleutherozoa': 3}
